<h1>DATA PREPROCESSING</h1>

<h2>EDA</h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt

In [ ]:
final_dataset=pd.read_csv('final_dataset2.csv')

In [ ]:
final_dataset

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

sw_list=stopwords.words('english')
word_count={
    "positive":[],
    "negative":[],
    "neutral":[],
    "irrelevant":[]
}

pattern=re.compile("[^\w ]")

for text,target in zip(final_dataset.text,final_dataset.sentiment):
    text=re.sub(pattern,"",text).lower().split()
    text=[word for word in text if word not in sw_list]
    word_count[target].extend(text)

In [ ]:
import seaborn as sns
fig, axes = plt.subplots(2, 2, figsize=(20,10.5))
for axis, (target, words) in zip(axes.flatten(), word_count.items()):
    bar_info = pd.Series(words).value_counts()[:25]
    sns.barplot(x=bar_info.values, y=bar_info.index, ax=axis)
    axis.set_title(f'Top words for {target}')
plt.show()

<h2>PREPROCESSING</h2>

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import emoji
import spacy
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import train_test_split
from sklearn.exceptions import NotFittedError

import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

In [ ]:
class Preprocessor:
    def __init__(self, stopwords=stopwords):
        self.vectorizer = TfidfVectorizer(lowercase=False, max_features=8000,
                                         min_df=10, ngram_range=(1, 3),
                                         tokenizer=None)
        self.stopwords = stopwords
        self.vectorizer_fitted = False
        
    def remove_urls(self, texts):
        print('Removing URLs...')
        pattern = re.compile('(\w+\.com ?/ ?.+)|(http\S+)')
        return [re.sub(pattern, '', text) for text in texts]
    
    def remove_double_space(self, texts):
        print('Removing double space...')
        pattern = re.compile(' +')
        return [re.sub(pattern, ' ', text) for text in texts]
        
    def remove_punctuation(self, texts):
        print('Removing Punctuation...')
        pattern = re.compile('[^a-z ]')
        return [re.sub(pattern, ' ', text) for text in texts]
    
    def remove_stopwords(self, texts):
        print('Removing stopwords...')
        return [[w for w in text.split(' ') if w not in self.stopwords] for text in tqdm(texts)]
    
    def remove_numbers(self, texts):
        print('Removing numbers...')
        return [' '.join([w for w in text if not w.isdigit()]) for text in tqdm(texts)]
    
    def decode_emojis(self, texts):
        print('Decoding emojis...')
        return [emoji.demojize(text, language='en') for text in texts] 
    
    def lemmatize(self, texts):
        print('Lemmatizing...')
        lemmatized_texts = []
        for text in tqdm(texts):
            doc = nlp(text)
            lemmatized_texts.append(' '.join([token.lemma_ for token in doc]))
                                    
        return lemmatized_texts
        
    def transform(self, X, y=None, mode='train'):
        X = X.copy()
        print('Removing Nans...')
        X = X[~X.isnull()]                          # delete nans
        X = X[~X.duplicated()]                      # delete duplicates
        
        if mode == 'train':
            self.train_idx = X.index
        else:
            self.test_idx = X.index
        print('Counting capitalized...')
        capitalized = [np.sum([t.isupper() for t in text.split()]) 
                           for text in np.array(X.values)]  # count capitalized
        # X['cap'] = capitalized
        print('Lowering...')
        X = [text.lower() for text in X]             # lower
        X = self.remove_urls(X)                      # remove urls
        X = self.remove_punctuation(X)               # remove punctuation
        X = self.remove_double_space(X)              # remove double space
        X = self.decode_emojis(X)                    # decode emojis
        X = self.remove_stopwords(X)                 # remove stopwords
        X = self.remove_numbers(X)                   # remove numbers                      
        X = self.lemmatize(X)                        # lemmatize
        
        if not self.vectorizer_fitted:
            self.vectorizer_fitted = True
            print('Fitting vectorizer...')
            self.vectorizer.fit(X)

        print('Vectorizing...')
        X = self.vectorizer.transform(X)             # vectorize
        
        return X

In [ ]:
pr=Preprocessor()

In [ ]:
data_train=final_dataset
y_final=final_dataset.sentiment

In [ ]:
data_train.text.info()

In [ ]:
y_final.info()

In [ ]:
data_train_pr = pr.transform(data_train['text'])
data_train_pr = pd.DataFrame.sparse.from_spmatrix(data_train_pr, columns=pr.vectorizer.get_feature_names_out())
y_final = y_final[y_final.index.isin(pr.train_idx)]
y_final.index = data_train_pr.index

In [ ]:
import pickle

In [ ]:
with open('data_train_pr.pkl','wb') as p:
    pickle.dump(data_train_pr,p)

with open('y_final.pkl','wb') as p:
    pickle.dump(y_final,p)

<h1>Training</h1>

In [ ]:
import pickle
with open('data_train_pr.pkl','rb') as p:
    X_data=pickle.load(p)
with open('y_final.pkl','rb') as p:
    y_data=pickle.load(p)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_data,y_data,test_size=0.4,random_state=0)

In [ ]:
X_train

In [ ]:
y_train

<h2> Logistic regression </h2>

In [ ]:
# from sklearn.metrics import classification_report
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.linear_model import LogisticRegression
# from scipy.stats import uniform, randint
# from sklearn.model_selection import train_test_split, GridSearchCV, KFold
# from sklearn.metrics import confusion_matrix

In [ ]:
# def train_cv(model, X_train, y_train, params, n_splits=5, scoring='f1_weighted'):
#     kf = KFold(n_splits=n_splits, random_state=0, shuffle=True)

#     cv = RandomizedSearchCV(model,
#                         params,
#                         cv=kf,
#                         scoring=scoring,
#                         return_train_score=True,
#                         n_jobs=-1,
#                         verbose=2,
#                         random_state=1
#                         )
#     cv.fit(X_train, y_train)

#     print('Best params', cv.best_params_)
#     return cv

In [ ]:
# rs_parameters = {
#     'penalty': ['l2', 'l1', 'elasticnet'],
#     'C': uniform(scale=10),
#     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
#     'l1_ratio': uniform(scale=10)
#     }

In [ ]:
# lr = LogisticRegression()
# model_cv_lr = train_cv(lr, X_train, y_train, rs_parameters)

# bestimator_lr = model_cv_lr.best_estimator_

In [ ]:
# print(classification_report(y_test, bestimator_lr.predict(X_test)))

In [ ]:
# from sklearn.metrics import confusion_matrix
# sns.heatmap(confusion_matrix(y_test, bestimator_lr.predict(X_test)), annot=True)
# plt.show()

<h2>Pytorch Lightning</h2>

In [ ]:
# import torch
# from torch import nn
# from torch.nn import functional as F
# from torch.utils.data import DataLoader
# from torch.utils.data import random_split
# import pytorch_lightning as pl
# import torch.utils.data as data_utils
# from torch.optim.lr_scheduler import StepLR
# from pytorch_lightning.callbacks import LearningRateMonitor
# from IPython.display import clear_output
# import numpy as np

# torch.manual_seed(42)

# def plot_loss(losses, title=''):
#     """
#     plots and saves loss progression while fitting
#     """
#     if len(losses) < 3:
#         return

#     pos = np.vstack(losses)
#     x, y = pos.T
#     plt.clf()
#     plt.ion()
#     plt.figure(figsize=(9, 5))
#     plt.plot(x, y)
#     plt.title(title)
#     clear_output(wait=True)
#     plt.show()

# class NNSentimentClassifier(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         self.softmax = nn.Softmax(dim=1)
#         self.dropout = nn.Dropout(0.2)
#         self.model = nn.Sequential(
#         nn.Linear(8000, 1000),
#         nn.ReLU(),
#         self.dropout,
#         nn.Linear(1000, 100),
#         nn.Tanh(),
#         self.dropout,
#         nn.Linear(100, 1000),
#         nn.ReLU(),
#         self.dropout,
#         nn.Linear(1000, 10),
#         nn.ReLU(),
#         self.dropout,
#         nn.Linear(10, 4)
#         )
#         self.acc_train_loss = []
#         self.acc_val_loss = []
    
#     def forward (self, x):
#         prediction = self.model(x)
#         return prediction
        
#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=0.0001)
#         scheduler = StepLR(optimizer, step_size=3, gamma=0.1, verbose=True)
#         return {"optimizer": optimizer, "lr_scheduler": scheduler}

#     def training_step(self, train_batch, batch_idx):
#         x, y = train_batch
#         y = y.long()
#         prediction = self.forward(x.float())
#         loss = F.cross_entropy(prediction, y)
#         self.log('train_loss', loss)
#         return loss
    
#     def validation_step(self, val_batch, batch_idx):
#         x, y = val_batch
#         y = y.long()
#         prediction = self.forward(x.float())
#         loss = F.cross_entropy(prediction, y)
#         self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
    
#     def test_step(self, test_batch, batc_idx):
#         x, y = test_batch
#         y = y.long()
#         prediction = model.forward(x.float())
#         preds = torch.argmax(prediction, dim=1)

#         return preds

In [ ]:
# # data
# target_to_idx = {
#     'Irrelevant': 0,
#     'Negative': 1,
#     'Neutral': 2,
#     'Positive': 3
#     }


# x_tensor=torch.from_numpy(X_train.to_numpy().astype(np.float64))
# y_tensor = torch.from_numpy(y_train.map(target_to_idx).values.astype(np.float64))
# train_data_tensor = data_utils.TensorDataset(x_tensor, y_tensor)
# val_size = int(len(train_data_tensor)*0.2)
# train_size = len(train_data_tensor)- int(len(train_data_tensor)*0.2)
# train_data_tensor, val_data_tensor = random_split(train_data_tensor, [train_size, val_size])
# train_loader = DataLoader(train_data_tensor, batch_size=10,num_workers=5)
# val_loader = DataLoader(val_data_tensor, batch_size=1,num_workers=5)

In [ ]:
# # model
# model = NNSentimentClassifier()
# # training

# trainer = pl.Trainer(gpus=1, precision=16, limit_train_batches=0.5, max_epochs=50)
# trainer.fit(model, train_loader, val_loader)

In [ ]:
# y_test_idx = torch.from_numpy(y_test.map(target_to_idx).values.astype(np.float64))
# test_data_tensor = data_utils.TensorDataset(torch.from_numpy(X_test.to_numpy().astype(np.float64)), y_test_idx)
# test_loader = DataLoader(test_data_tensor, batch_size=1)

<h2>LSTM</h2>

In [24]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import SGD
import keras.backend as K
from sklearn.preprocessing import StandardScaler

In [ ]:
vocab_size = 5000
embedding_size = 32
epochs=20
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model= Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=8000))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))